In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from data.dataset import SkeletonDataset
from torch_geometric.data import DataLoader
from models.net import DualGraphTransformer
from torch.autograd import Variable
from einops import rearrange
from args import make_args
from optimizer import get_std_opt
from torch_scatter import scatter

In [2]:
train_dataset= SkeletonDataset(root="/home/project/gcn/Apb-gcn/NTU-RGB+D", name='cv_val', benchmark='cv', sample = 'val')

In [3]:
train_dataset.labels = train_dataset.labels.to(torch.device('cuda:0'))

In [4]:
train_loader = DataLoader(train_dataset.data, batch_size = 4)
#val_loader = DataLoader(val_dataset, batch_size = 32)

In [5]:
model = DualGraphTransformer(in_channels = 7, hidden_channels = 16, out_channels = 16, num_layers = 4, num_heads = 8)

In [6]:
model = model.to(torch.device('cuda:0'))

In [21]:
for batch_idx, batch in enumerate(train_loader):
    batch = batch.to(torch.device('cuda:0'))
    target = train_dataset.labels.gather(0, batch.batch)

    train_dataset.skeleton_ = train_dataset.skeleton_.to(torch.device('cuda:0'))
    print(batch.x.shape)
    if torch.isnan(batch.x).any():
        print(batch.x)
    output = model(batch.x, adj=train_dataset.skeleton_)
        
    #print(output)
    loss = nn.functional.cross_entropy(output, target.long())
    print(loss)
    pred = torch.max(output, 1)[1]
    #print(pred)
    results = pred == target
    correct_points = torch.sum(results.long())
    acc = correct_points.float()/results.size()[0]
    #print(acc)

torch.Size([379, 25, 7])
tensor(4.1446, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([607, 25, 7])
tensor(4.1048, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([407, 25, 7])
tensor(4.0200, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([678, 25, 7])
tensor(4.1118, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([401, 25, 7])
tensor(4.1836, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([492, 25, 7])
tensor(4.1464, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([374, 25, 7])
tensor(4.0969, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([333, 25, 7])
tensor(4.1243, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([365, 25, 7])
tensor(4.1098, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([339, 25, 7])
tensor(4.2247, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([424, 25, 7])
tensor(4.2079, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([475, 25, 7])
tensor(4.1067, device='cuda:0', grad_fn=<NllLossBac

tensor(4.0859, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([386, 25, 7])
tensor(4.0634, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([353, 25, 7])
tensor(4.1103, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([486, 25, 7])
tensor(4.1143, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([399, 25, 7])
tensor([[[ 0.1369, -0.2093,  4.1838,  ...,  1.5708,  1.5708,  4.1912],
         [ 0.1404,  0.0435,  4.2062,  ...,  1.5708,  1.5708,  4.2087],
         [ 0.1460,  0.2950,  4.2192,  ...,  1.5708,  1.5708,  4.2320],
         ...,
         [-0.0788, -0.3723,  4.0697,  ...,  1.5708,  1.5708,  4.0874],
         [ 0.4338, -0.3719,  4.3485,  ...,  1.5708,  1.5708,  4.3859],
         [ 0.4383, -0.3433,  4.3341,  ...,  1.5708,  1.5708,  4.3697]],

        [[ 0.5979,  0.0329,  1.2015,  ...,  1.2203,  1.3894,  1.3424],
         [ 0.5922,  0.2499,  1.1701,  ...,  1.2256,  1.4155,  1.3350],
         [ 0.5805,  0.4524,  1.1256,  ...,  1.2418,  1.4535,  1.3449],
         

KeyboardInterrupt: 